# Step 1: Develop a Machine Learning Algorithm

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

df_drug = pd.read_csv(r"C:\Users\delva\OneDrive\Desktop\Data Science\Projects\datasets\drug200.csv")
label_encoder = LabelEncoder()

categorical_features = [feature for feature in df_drug.columns if df_drug[feature].dtypes == 'O']
for feature in categorical_features:
    df_drug[feature] = label_encoder.fit_transform(df_drug[feature])

X = df_drug.drop("Drug", axis = 1)
y = df_drug["Drug"]

X_train,X_valid,y_train,y_valid = train_test_split(X,y,
                                                  train_size = 0.8, test_size = 0.2,
                                                  random_state = 0)

#model = DecisionTreeClassifier(criterion = "entropy")
model = RandomForestClassifier(random_state = 0)
model.fit(X_train,y_train)

kfold = KFold(n_splits = 10,random_state = 0, shuffle = True)
cv_results = cross_val_score(model,X,y, cv=kfold, scoring = "accuracy")
print(cv_results.mean(), cv_results.std())

0.99 0.020000000000000018


In [2]:
df_drug.columns

Index(['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K', 'Drug'], dtype='object')

In [3]:
import pickle
pickle_file = open('model.pkl', 'ab')
pickle.dump(model,pickle_file)
pickle_file.close()

# Step 2: Make an individual prediction from the trained model.

In [4]:
df_drug = pd.read_csv(r"C:\Users\delva\OneDrive\Desktop\Data Science\Projects\datasets\drug200.csv")
label_encoder = LabelEncoder()
categorical_features = [feature for feature in df_drug.columns if df_drug[feature].dtypes == 'O']
for feature in categorical_features:
    print(feature, list(df_drug[feature].unique()), list(label_encoder.fit_transform(df_drug[feature].unique())),"\n")

Sex ['F', 'M'] [0, 1] 

BP ['HIGH', 'LOW', 'NORMAL'] [0, 1, 2] 

Cholesterol ['HIGH', 'NORMAL'] [0, 1] 

Drug ['DrugY', 'drugC', 'drugX', 'drugA', 'drugB'] [0, 3, 4, 1, 2] 



In [5]:
gender_map = {"F": 0, "M": 1}
bp_map = {"HIGH": 0, "LOW": 1, "NORMAL": 2}
cholesterol_map = {"HIGH": 0, "NORMAL": 1}
drug_map = {0: "Drug Y", 3: "Drug C", 4: "Drug X", 1: "Drug A", 2: "Drug B"}

def predict_drug(Age,Sex,BP,Cholesterol,Na_to_K):
    # 1. Read the machine learning model from its saved state
    pickle_file = open('model.pkl', 'rb')
    model = pickle.load(pickle_file)
    
    #2 Transform the "raw data" passed into the function to the encoded /numerical values using the maps / dictionaries
    Sex = gender_map[Sex]
    BP = bp_map[BP]
    Cholesterol = cholesterol_map[Cholesterol]
    
    #3 Make an individual prediction for this set of data
    y_predict = model.predict([[Age, Sex,BP,Cholesterol, Na_to_K]])[0]
    
   #4 Return the "raw" version of the prediction i.e the actual name of the drug rather than the numerical encoded version
    return drug_map[y_predict]

In [6]:
#Prediction 1
predict_drug(47,"F","LOW","HIGH",14)

C:\Users\delva\anaconda3\lib\site-packages\sklearn\base.py:446: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


'Drug C'

In [7]:
#Prediction 2
predict_drug(60,"F","LOW","HIGH",20)

C:\Users\delva\anaconda3\lib\site-packages\sklearn\base.py:446: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


'Drug Y'

# Step 3: Develop a Web Service Wrapper

In [8]:
from flask import Flask

import pickle
from flask import request, jsonify

app =Flask(__name__)

gender_map = {"F": 0, "M": 1}
bp_map = {"HIGH": 0, "LOW": 1, "NORMAL":2}
cholesterol_map = {"HIGH": 0, "NORMAL": 1}
drug_map = {0:"DRUG Y", 3: "DRUG C", 4: "DRUG X", 1:"DRUG A", 2: "DRUG  B"}

def predict_drug(Age,Sex,BP,Cholesterol,Na_to_K):
    #1 Read the machine learning model from its saved state
    pickle_file = open('model.pkl', 'rb')
    model = pickle.load(pickle_file)
    #2 Transform the "raw data" passed into the function to the encoded / numerical values using the maps / dictionaries
    Sex = gender_map[Sex]
    BP = bp_map[BP]
    Cholesterol = cholesterol_map[Cholesterol]
    
    #3 Make an individual prediction for this set of data
    y_predict = model.predict([[Age,Sex,BP,Cholesterol,Na_to_K]])[0]
    
    #4 Return the "raw" version of the prediction i.e the actual name of the drug rather than the numerical encoded version
    return drug_map[y_predict]

@app.route("/")
def hello():
    return "A test web service for accessing a machine learning model to make drug reccommendations v2."

@app.route('/drug', methods = ['GET'])
def api_all():
    
    Age = int(request.args['Age'])
    Sex = request.args['Sex']
    BP = request.args['BP']
    Cholesterol = request.args['Cholesterol']
    Na_to_K = float(request.args['Na_to_K'])
    
    drug = predict_drug(Age,Sex,BP,Cholesterol,Na_to_K)
    
    #return(jsonify(drug))
    return(jsonify(recommended_drug = drug))

In [9]:
import requests
from ipywidgets import Label, BoundedFloatText, BoundedIntText, Dropdown, Button, Output, VBox

prescribe_label = Label('Drug prescription prediction for age, gender, bp, cholesterol and Na to K')
age_text = BoundedIntText(min=16, max=100, value=47, description="Age:", disabled=False)
gender_dropdown = Dropdown(options=['F', 'M'], description='Gender:', disabled=False)
bp_dropdown = Dropdown(options=['HIGH', 'LOW', 'NORMAL'], value="LOW", description='BP:', disabled=False)
cholesterol_dropdown = Dropdown(options=['HIGH', 'NORMAL'], description='Cholesterol:', disabled=False)
na_to_k_text = BoundedFloatText(min=0.0, max=50.0, value=14, description="Na to K", disabled=False)
prescribe_button = Button(description="Prescribe")
prescribe_output = Output()

# Button click event handlers ...
def prescribe_button_on_click(b):
    
    request_url = f"http://127.0.0.1:5000/drug?Age={age_text.value}&Sex={gender_dropdown.value}&BP={bp_dropdown.value}&Cholesterol={cholesterol_dropdown.value}&Na_to_K={na_to_k_text.value}"
    response = requests.get(request_url)
    recommended_drug = response.json()["recommended_drug"]

    prescribe_output.clear_output()
    with prescribe_output:

        print(f"The recommended drug is {recommended_drug}")
        
prescribe_button.on_click(prescribe_button_on_click)

vbox_prescribe = VBox([prescribe_label, age_text, gender_dropdown, bp_dropdown, cholesterol_dropdown, na_to_k_text, prescribe_button, prescribe_output])

vbox_prescribe

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /drug?Age=23&Sex=M&BP=NORMAL&Cholesterol=NORMAL&Na_to_K=14.0 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000028965360088>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /drug?Age=23&Sex=M&BP=NORMAL&Cholesterol=NORMAL&Na_to_K=14.0 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000028965352C88>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))